In [2]:
import pandas as pd
import os
import random
import torch
import pickle

In [3]:
random.seed(2024)
!ls

ablation_20.csv     lambda_1.csv	     processed_data.csv
ablation_50.csv     lambda.csv		     readme.md
ablation.csv	    pcgrad_1.csv	     Statistics_KuaiRec.ipynb
data		    pcgrad.csv		     table_50.csv
dif_steps_1.csv     PE_1.csv		     table.csv
dif_steps.csv	    PE.csv		     table.ipynb
formatted_data.csv  preprocess_amazon.ipynb  table_new_50.csv
item_class.ipynb    processed_50.csv	     table_new.csv


# Load original dataset

### Selecting datasets

In [12]:

dataset_name =  "yelp"
root = 'data/'+dataset_name+'/'
path = root + dataset_name + '.inter'
dataset = pd.read_csv(path, header=0,sep='\t', skipinitialspace=True,usecols=[0,1,2,3])
# path = root + dataset_name + '.csv'
# dataset = pd.read_csv(path, header=0,sep=',', skipinitialspace=True)
# print(dataset.head(2))
# if dataset_name ==  "mind":
#     dataset=dataset.drop('label:float', axis=1)
print(dataset.shape)
dataset.head()

(5261669, 4)


,user_id:token,item_id:token,rating:float,timestamp:float
0,bv2nCi5Qv5vroFiqKGopiw,0W4lkclzZThpx3V65bVgig,5,1464364800
1,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,1464364800
2,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,1464364800
3,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,1464364800
4,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,1464364800


In [15]:

dataset.columns = ['user_id','item_id', 'rating','timestamp']
dataset = dataset[dataset.rating > 3]
# dataset.columns = ['user_id', 'item_id',  'timestamp']
dataset.head()

,user_id,item_id,rating,timestamp
0,bv2nCi5Qv5vroFiqKGopiw,0W4lkclzZThpx3V65bVgig,5,1464364800
1,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,1464364800
2,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,1464364800
3,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,1464364800
4,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,1464364800


### filter by number of interactions

In [16]:
user_threshold = 5
item_threshold = 5


filtered_dataset = dataset.copy()
while(True):
    ori_len = len(filtered_dataset)
    print(ori_len)
    filtered_dataset = filtered_dataset[filtered_dataset['user_id'].map(filtered_dataset['user_id'].value_counts()) >= user_threshold]
    filtered_dataset = filtered_dataset[filtered_dataset['item_id'].map(filtered_dataset['item_id'].value_counts()) >= item_threshold]
    print(len(filtered_dataset))
    # break
    if len(filtered_dataset) == ori_len:
        break
print('done!')
# filtered_dataset.head()

3476664
1923578
1923578
1862100
1862100
1857504
1857504
1857112
1857112
1857020
1857020
1856992
1856992
1856992
done!


# Map all ids

In [17]:

all_user = filtered_dataset.user_id
all_item = filtered_dataset.item_id

In [18]:
user_id, user_token = pd.factorize(all_user)
print("num user: ",len(user_token))
item_id, item_token = pd.factorize(all_item)
print("num item: ", len(item_token))
user_mapping_dict = {_: idx for idx, _ in enumerate(user_token)}
item_mapping_dict = {_: idx+1 for idx, _ in enumerate(item_token)}

num user:  136346
num item:  64669


In [19]:
mapped_dataset = filtered_dataset.copy()
mapped_dataset['user_id'] = filtered_dataset['user_id'].apply(lambda x: user_mapping_dict[x])
mapped_dataset['item_id'] = filtered_dataset['item_id'].apply(lambda x: item_mapping_dict[x])
mapped_dataset.head()

,user_id,item_id,rating,timestamp
0,0,1,5,1464364800
1,0,2,5,1464364800
2,0,3,5,1464364800
3,0,4,5,1464364800
4,0,5,4,1464364800


In [20]:
dataset = mapped_dataset.sort_values(by=['user_id', 'timestamp'])
user_group = dataset.groupby('user_id')['item_id'].apply(list)
user_group.head()

user_id
0                                   [1, 2, 3, 4, 5, 6]
1    [17, 32, 10, 43, 14, 36, 34, 20, 1, 37, 42, 11...
2    [63, 73, 49, 54, 58, 693, 75, 691, 71, 690, 47...
3    [156, 92, 140, 144, 131, 95, 120, 134, 149, 24...
4    [183, 189, 520, 258, 174, 175, 521, 227, 229, ...
Name: item_id, dtype: object

In [21]:
k=0
for i in user_group:
    k+=len(i)
print(k)
print(k/len(user_group))

1856992
13.619702814897394


### In-domain sequences

In [22]:
# max_seq_len = 20
# 
# def truncate_or_pad(seq):
#     cur_seq_len = len(seq)
#     if cur_seq_len > max_seq_len:
#         return seq[-max_seq_len:], max_seq_len
#     else:
#         return seq + [PAD] * (max_seq_len - cur_seq_len), cur_seq_len

# 
# dataset = mapped_dataset.sort_values(by=['user_id', 'timestamp'])
# user_group = dataset.groupby('user_id')['item_id'].apply(list)
train, val, test = [], [], []

for user_id, user_seq in list(zip(user_group.index, user_group.tolist())):
    # iterate on each user sequence
    # ------ test sample ------------
    test_data = user_seq[-1]
    test.append([test_data])
    # ------ val sample -------------
    val_data = user_seq[-2]
    val.append([val_data])
    # ------ train sample -----------
    train_data= user_seq[:-2]
    train.append(train_data)
data = {'train': train, 'val': val, 'test': test}
with open(os.path.join(root, 'dataset.pkl'), 'wb') as f:
    pickle.dump(data, f)
# torch.save(train, os.path.join(output_path, 'dataset.pkl'))

print('done!')

done!


In [173]:
train_dataset = data['train']
print(len(train_dataset))
max_len=50
# def split_onebyone(data):
# # 分割成了最小长度为2的交互递增序列
#     id_seq = {}
#     idx = 0
#     for seq_temp in data:
#         #只能从预截取长度中进行切分，不然属于利用了额外数据！！！
#         for star in range(max_len):
#             id_seq[idx] = seq_temp[-star-2:]
#             idx += 1
#             if star+2 > len(seq_temp):
#                 break
#     return id_seq
def split_onebyone(u2seq):
# 分割成了最小长度为2的交互递增序列
    id_seq = {}
    idx = 0

    for seq_temp in u2seq:
        seq_temp = seq_temp[-max_len-1:]
        for star in range(len(seq_temp)-1):
            id_seq[idx] = seq_temp[:star+2]
            idx += 1
    return id_seq

6034


In [174]:
split_train = split_onebyone(train_dataset)
len(split_train)

231400

### mixed sequence

# Save

In [12]:
# for d_name, d in zip(output_dataset_name_list, mapped_dataset_list):
#     d.to_csv(os.path.join(output_path, d_name, 'inter.csv'), sep=',', index=None)